#### Develop an agent function to process a list of WhatsApp messages and extract events.

1. Load the WhatsApp messages from a file.
2. Process the messages to extract events.
3. Visualize the events in a table.

In [23]:
# Libraries & functions to pretty print text and JSON responses, just needed for the notebook
import textwrap # Text wrapping
from pprint import pprint # Pretty printing json

def pretty_print(response):
    if isinstance(response, (str, dict)):
        print(textwrap.fill(response, width=80))
    else:
        response_dict = response.model_dump()
        pprint(response_dict, width=80)

In [24]:
# Open the file

with open('dummy-data/dummy_whatsapp_5_events.txt', 'r', encoding='utf-8') as file:
    sample_message_history = file.read()


In [25]:
pretty_print(sample_message_history)

[22/11/24, 10:11:31] Sofia: We played on Wednesday at the usual spot. Good
memories! [22/11/24, 10:25:31] Maria: Yes, it was fun! 👍 [22/11/24, 10:36:31]
Ana: Yes, it was fun! 👍 [22/11/24, 10:43:31] Laura: Yes, it was fun! 👍
[22/11/24, 10:50:31] Carmen: Yes, it was fun! 👍 [22/11/24, 10:53:31] Julia:
Missed it 😢 [22/11/24, 11:07:31] Elena: Yes, it was fun! 👍 [22/11/24, 11:20:31]
Isabel: So sad I couldn't make it. [22/11/24, 11:26:31] Paula: Yes, it was fun!
👍 [22/11/24, 11:40:31] Victoria: Yes, it was fun! 👍 [01/01/25, 10:06:31] Laura:
Who's up for a match? [01/01/25, 10:11:31] Laura: How about tomorrow at 17:00 at
the complex? [01/01/25, 10:25:31] Maria: Count me in! 👍 [01/01/25, 10:32:31]
Sofia: Count me in! 👍 [01/01/25, 10:47:31] Ana: Count me in! 👍 [01/01/25,
10:55:31] Carmen: Count me in! 👍 [01/01/25, 11:01:31] Julia: Count me in! 👍
[01/01/25, 11:05:31] Elena: Count me in! 👍 [01/01/25, 11:20:31] Isabel: Count me
in! 👍 [01/01/25, 11:31:31] Paula: Count me in! 👍 [01/01/25, 11:40:31] V

In [26]:
from pydantic import BaseModel, Field
from openai import OpenAI
from dotenv import load_dotenv
from prompts import SYSTEM_PROMPT

load_dotenv()

client = OpenAI()

class CalendarEvent(BaseModel):
    name: str = Field(description="The name of the event")
    event_date: str = Field(description="The date of the event. Not the date the message timestamp.")
    time: str = Field(description="The time of the event")
    participants: list[str] = Field(description="The participants of the event")
    number_of_participants: int = Field(description="The number of participants of the event")
    not_attending: list[str] = Field(description="The people who are not attending the event")
    didnt_confirm: list[str] = Field(description="The people who didn't confirm the event")
    location: str = Field(description="The location of the event")

class CalendarEvents(BaseModel):
    events: list[CalendarEvent]

completion = client.beta.chat.completions.parse(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": sample_message_history},
    ],
    response_format=CalendarEvents,
)

response = completion.choices[0].message.parsed

In [27]:
pretty_print(completion.choices[0].message)

{'audio': None,
 'content': '{"events":[{"name":"Game on '
            'Wednesday","event_date":"20/11/24","time":"N/A","participants":["Sofia","Maria","Ana","Laura","Carmen","Elena","Paula","Victoria"],"number_of_participants":8,"not_attending":["Julia","Isabel"],"didnt_confirm":[],"location":"Usual '
            'spot"},{"name":"Match on '
            '02/01/25","event_date":"02/01/25","time":"17:00","participants":["Laura","Maria","Sofia","Ana","Carmen","Julia","Elena","Isabel","Paula","Victoria"],"number_of_participants":10,"not_attending":[],"didnt_confirm":[],"location":"Complex"},{"name":"Game '
            'on '
            'Friday","event_date":"03/01/25","time":"19:00","participants":["Victoria","Maria","Sofia","Ana","Laura","Carmen","Elena"],"number_of_participants":7,"not_attending":["Julia","Paula"],"didnt_confirm":[],"location":"Indoor '
            'court"},{"name":"Game on '
            'Sunday","event_date":"04/01/25","time":"18:00","participants":["Maria","Sofia","Ana

In [28]:
response.events

[CalendarEvent(name='Game on Wednesday', event_date='20/11/24', time='N/A', participants=['Sofia', 'Maria', 'Ana', 'Laura', 'Carmen', 'Elena', 'Paula', 'Victoria'], number_of_participants=8, not_attending=['Julia', 'Isabel'], didnt_confirm=[], location='Usual spot'),
 CalendarEvent(name='Match on 02/01/25', event_date='02/01/25', time='17:00', participants=['Laura', 'Maria', 'Sofia', 'Ana', 'Carmen', 'Julia', 'Elena', 'Isabel', 'Paula', 'Victoria'], number_of_participants=10, not_attending=[], didnt_confirm=[], location='Complex'),
 CalendarEvent(name='Game on Friday', event_date='03/01/25', time='19:00', participants=['Victoria', 'Maria', 'Sofia', 'Ana', 'Laura', 'Carmen', 'Elena'], number_of_participants=7, not_attending=['Julia', 'Paula'], didnt_confirm=[], location='Indoor court'),
 CalendarEvent(name='Game on Sunday', event_date='04/01/25', time='18:00', participants=['Maria', 'Sofia', 'Ana', 'Carmen', 'Elena', 'Julia'], number_of_participants=6, not_attending=['Laura'], didnt_con

In [29]:
for event in response.events:
    print(event.model_dump_json())

{"name":"Game on Wednesday","event_date":"20/11/24","time":"N/A","participants":["Sofia","Maria","Ana","Laura","Carmen","Elena","Paula","Victoria"],"number_of_participants":8,"not_attending":["Julia","Isabel"],"didnt_confirm":[],"location":"Usual spot"}
{"name":"Match on 02/01/25","event_date":"02/01/25","time":"17:00","participants":["Laura","Maria","Sofia","Ana","Carmen","Julia","Elena","Isabel","Paula","Victoria"],"number_of_participants":10,"not_attending":[],"didnt_confirm":[],"location":"Complex"}
{"name":"Game on Friday","event_date":"03/01/25","time":"19:00","participants":["Victoria","Maria","Sofia","Ana","Laura","Carmen","Elena"],"number_of_participants":7,"not_attending":["Julia","Paula"],"didnt_confirm":[],"location":"Indoor court"}
{"name":"Game on Sunday","event_date":"04/01/25","time":"18:00","participants":["Maria","Sofia","Ana","Carmen","Elena","Julia"],"number_of_participants":6,"not_attending":["Laura"],"didnt_confirm":["Victoria","Isabel"],"location":"Indoor court"}

In [30]:
import pandas as pd
import json

pd.DataFrame([json.loads(event.model_dump_json()) for event in response.events])


,name,event_date,time,participants,number_of_participants,not_attending,didnt_confirm,location
0,Game on Wednesday,20/11/24,N/A,"[Sofia, Maria, Ana, Laura, Carmen, Elena, Paul...",8,"[Julia, Isabel]",[],Usual spot
1,Match on 02/01/25,02/01/25,17:00,"[Laura, Maria, Sofia, Ana, Carmen, Julia, Elen...",10,[],[],Complex
2,Game on Friday,03/01/25,19:00,"[Victoria, Maria, Sofia, Ana, Laura, Carmen, E...",7,"[Julia, Paula]",[],Indoor court
3,Game on Sunday,04/01/25,18:00,"[Maria, Sofia, Ana, Carmen, Elena, Julia]",6,[Laura],"[Victoria, Isabel]",Indoor court
4,"Game on January 31st, 2025",31/01/25,N/A,"[Victoria, Maria, Sofia, Ana, Carmen, Julia, E...",8,"[Laura, Isabel]",[],Complex


In [31]:
# process the messages to extract events
class CalendarEvent(BaseModel):
    name: str
    date: str
    time: str
    participants: str
    location: str
    number_of_participants: int

class CalendarEvents(BaseModel):
    events: list[CalendarEvent]

def process_messages(message_history):
    
    # Initialize the OpenAI client
    client = OpenAI()

    # Make the OpenAI API call to extract the events
    completion = client.beta.chat.completions.parse(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": sample_message_history},
        ],
        response_format=CalendarEvents,
    )

    # Parse the response
    response = completion.choices[0].message.parsed

    return pd.DataFrame([json.loads(event.model_dump_json()) for event in response.events])


In [32]:
events_df = process_messages(sample_message_history)

events_df

,name,date,time,participants,location,number_of_participants
0,Game on Wednesday,20/11/24,N/A,"Sofia, Maria, Ana, Laura, Carmen, Elena, Paula...",Usual spot,8
1,Upcoming match,02/01/25,17:00,"Laura, Maria, Sofia, Ana, Carmen, Julia, Elena...",Complex,10
2,Game on Friday,03/01/25,19:00,"Victoria, Maria, Sofia, Ana, Carmen, Elena",Indoor court,6
3,Game on Sunday,05/01/25,18:00,"Maria, Sofia, Ana, Carmen, Julia, Elena",Indoor court,6
4,Game on 2025-01-31,31/01/25,N/A,"Victoria, Maria, Sofia, Ana, Carmen, Julia, El...",Complex,8


In [33]:
from datetime import datetime

datetime.now().strftime("%A")

'Thursday'